In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)
        self.bn1 = nn.BatchNorm2d(num_features=16, eps=1e-05)
        self.conv2 = nn.Conv2d(16, 16, 3)
        self.bn2 = nn.BatchNorm2d(num_features=16, eps=1e-05)
        self.conv3 = nn.Conv2d(16, 16, 3)
        self.bn3 = nn.BatchNorm2d(num_features=16, eps=1e-05)
        # self.conv4 = nn.Conv2d(32, 64, 3)
        # self.bn4 = nn.BatchNorm2d(num_features=64, eps=1e-05)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.trans1 = nn.Conv2d(16, 8, 1)
        self.bnt1 = nn.BatchNorm2d(num_features=8, eps=1e-05)

        self.conv5 = nn.Conv2d(8, 32, 3)
        self.bn5 = nn.BatchNorm2d(num_features=32, eps=1e-05)
        self.conv6 = nn.Conv2d(32, 32, 3)
        self.bn6 = nn.BatchNorm2d(num_features=32, eps=1e-05)
        self.conv7 = nn.Conv2d(32, 32, 3)
        self.bn7 = nn.BatchNorm2d(num_features=32, eps=1e-05)
        self.conv8 = nn.Conv2d(64, 128, 3)
        self.bn8 = nn.BatchNorm2d(num_features=64, eps=1e-05)

        self.gap = nn.AvgPool2d(kernel_size = 7)
        self.trans2 = nn.Conv2d(32, 10, 1)
        self.drop = nn.Dropout(p=0.1)
        self.fc = nn.Linear(32,10)

    def forward(self, x):
        x = F.relu(self.bn2(self.conv2(self.drop(F.relu(self.bn1(self.conv1(x)))))))
        # x = self.drop(x)

        x = self.pool1(F.relu(self.bn3(self.conv3(x))))
        # x = self.drop(x)
        x = F.relu(self.bnt1(self.trans1(x)))

        x = self.bn6(self.conv6(self.drop(F.relu(self.bn5(self.conv5(x))))))
        # x = self.drop(x)
        x = self.gap(x)

        # x = self.trans2(x)

        # x = x.view(-1, 10)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
           Dropout-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           2,320
       BatchNorm2d-5           [-1, 16, 24, 24]              32
            Conv2d-6           [-1, 16, 22, 22]           2,320
       BatchNorm2d-7           [-1, 16, 22, 22]              32
         MaxPool2d-8           [-1, 16, 11, 11]               0
            Conv2d-9            [-1, 8, 11, 11]             136
      BatchNorm2d-10            [-1, 8, 11, 11]              16
           Conv2d-11             [-1, 32, 9, 9]           2,336
      BatchNorm2d-12             [-1, 32, 9, 9]              64
          Dropout-13             [-1, 32, 9, 9]               0
           Conv2d-14             [-1, 3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:


torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0803, Accuracy: 9782/10000 (98%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0615, Accuracy: 9826/10000 (98%)



  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0289, Accuracy: 9913/10000 (99%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0254, Accuracy: 9922/10000 (99%)



loss=0.019826747477054596 batch_id=937: 100%|██████████| 938/938 [00:16<00:00, 57.07it/s]



Test set: Average loss: 0.0241, Accuracy: 9935/10000 (99%)

